In [ ]:

from langchain_openai import ChatOpenAI
from langgraph.func import task, entrypoint
from langgraph.checkpoint.memory import MemorySaver


@task
def generate_response(user_query: str) -> str:
    llm = ChatOpenAI(model="Qwen/Qwen3-8B", temperature=0)
    response = llm.invoke(user_query)
    return response.content


@entrypoint(checkpointer=MemorySaver())
def chatbot_workflow(query: str) -> str:
    agent_response = generate_response(query)
    return agent_response.result()


config = {"configurable": {"thread_id": "chatbot_1"}}
response = chatbot_workflow.invoke("你好，你好吗？", config=config)
response
# 你好！我很好，谢谢关心！😊 你呢？今天过得怎么样？有什么我可以帮你的吗？'


In [ ]:
from langgraph.func import task, entrypoint
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.store.base import BaseStore
from langgraph.types import StreamWriter
from langchain_core.runnables import RunnableConfig
from typing import Any


@entrypoint(checkpointer=MemorySaver(), store=InMemoryStore())
def my_workflow(
    user_input: dict,
    *,
    previous: Any = None,
    store: BaseStore,
    writer: StreamWriter,
    config: RunnableConfig,
) -> str:
    api_key_from_config = config.get("configurable", {}).get("api_version")
    writer(
        f"工作流{config.get('metadata', {}).get('thread_id')}，以API版本\
            启动：{api_key_from_config}"
    )

    # 使用注入参数的工作流逻辑
    return f"工作流处理的输入:{user_input}"


# 示例调用
result = my_workflow.invoke(
    {"messages": "hello"}, config={"configurable": {"thread_id": "complex_workflow_1"}}
)
result
# "工作流处理的输入:{'messages': 'hello'}"